<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#User-inputs" data-toc-modified-id="User-inputs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>User inputs</a></span><ul class="toc-item"><li><span><a href="#Input-data-file" data-toc-modified-id="Input-data-file-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Input data file</a></span></li><li><span><a href="#Details-on-the-size-reduction-to-perform-via-PCA" data-toc-modified-id="Details-on-the-size-reduction-to-perform-via-PCA-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Details on the size reduction to perform via PCA</a></span></li><li><span><a href="#Clustering:-number-of-clusters-to-generate-for-each-final-attribute" data-toc-modified-id="Clustering:-number-of-clusters-to-generate-for-each-final-attribute-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Clustering: number of clusters to generate for each final attribute</a></span></li><li><span><a href="#Polyhedral-uncertainty-set-parameters" data-toc-modified-id="Polyhedral-uncertainty-set-parameters-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Polyhedral uncertainty set parameters</a></span></li><li><span><a href="#Scenarios-definition" data-toc-modified-id="Scenarios-definition-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Scenarios definition</a></span></li></ul></li><li><span><a href="#Computation" data-toc-modified-id="Computation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Computation</a></span></li><li><span><a href="#Querying-the-ouput-results:-example" data-toc-modified-id="Querying-the-ouput-results:-example-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Querying the ouput results: example</a></span></li></ul></div>

# Import packages

In [1]:
import sys  
print(sys.executable)
sys.path.insert(0, 'src/')

import pca_size_reduction as psr
import polyhedral_uncertainty_set as pus
import clustering_analysis as cl
import analysis_plots as ap
import scenarios_pus as spus

/Users/nvdbconsulting/Documents/local_research/uncertainty_modelling_for_energy/myenv/bin/python3.11


# User inputs
Here are all the inputs that the user must provide and cacn play with to generate different scenarios and associated describing linear inequality constraints

## Input data file

In [2]:
# File path containing the data
the_input_file = 'data/energy_UK_2015_dataset.csv'

# Number of values per data points
the_n_values_per_attribute_per_data_point = 24

## Details on the size reduction to perform via PCA 

In [3]:
######################
# PCA size reduction #
######################

###########
# Level 1 #
###########

# the explained variance threshold
the_explained_variance_threshold_level_1 = 5*10.0**-5

# the maximum number of PCA components to retain
the_n_directions_threshold_level_1 = None

# Set which attributes should be merged together in the 1st layer of PCA size reduction
the_attributes_to_merge_pca_level_1 = {
    'Elec':['Elec'],
    'GasDem':['GasDem_EA', 'GasDem_EM', 'GasDem_NE', 'GasDem_NO', 'GasDem_NT', 'GasDem_NW', 'GasDem_SC', 'GasDem_SE', 'GasDem_SO', 'GasDem_SW', 'GasDem_WM'],
    'Temp':['Temp_EA', 'Temp_EM', 'Temp_NE', 'Temp_NO', 'Temp_NT', 'Temp_NW', 'Temp_SC', 'Temp_SE', 'Temp_SO', 'Temp_SW', 'Temp_WM', 'Temp_WN', 'Temp_WS'],
    'Solar':['Solar_EA', 'Solar_EM', 'Solar_NE', 'Solar_NO','Solar_NT', 'Solar_NW', 'Solar_SC', 'Solar_SE', 'Solar_SO', 'Solar_SW', 'Solar_WM', 'Solar_WN', 'Solar_WS'],
    'WindOff':['WindOff_EA', 'WindOff_EM', 'WindOff_NE', 'WindOff_NO', 'WindOff_NT', 'WindOff_NW', 'WindOff_SC', 'WindOff_SE', 'WindOff_SO', 'WindOff_SW', 'WindOff_WN', 'WindOff_WS'],
    'WindOn':['WindOn_EA', 'WindOn_EM', 'WindOn_NE', 'WindOn_NO', 'WindOn_NT', 'WindOn_NW', 'WindOn_SC', 'WindOn_SE', 'WindOn_SO', 'WindOn_SW', 'WindOn_WM', 'WindOn_WN', 'WindOn_WS']
}

###########
# Level 2 #
###########

# the explained variance threshold
the_explained_variance_threshold_level_2 = 10.0**-4

# the maximum number of PCA components to retain
the_n_directions_threshold_level_2 = 48

# Set which attributes should be merged together in the 2nd layer of PCA size reduction
the_attributes_to_merge_pca_level_2 = {
    'Seasonality':['Elec','GasDem','Temp'],
    'Solar':['Solar'],
    'Wind':['WindOn','WindOff']
}

## Clustering: number of clusters to generate for each final attribute

In [4]:
the_n_clusters = {'Seasonality': 4, 'Solar': 4, 'Wind': 4}
the_clustering_method = 'kmeans' # Choose between 'kmeans', 'gmm', and 'dpgmm'

## Polyhedral uncertainty set parameters

In [5]:
the_details_polyhedral_uncertainty_set_generation = {
        'Seasonality': {'α':0.025,
                        'cumulated_budget':20.0,
                        'pairwise_budget':1.5,
                        'n_dir_pairwise_budget':10},
        'Solar': {'α':0.025,
                  'cumulated_budget':20.0,
                  'pairwise_budget':1.5,
                  'n_dir_pairwise_budget':10},
        'Wind': {'α':0.025,
                 'cumulated_budget':20.0,
                 'pairwise_budget':1.5,
                 'n_dir_pairwise_budget':10}
        }

## Scenarios definition

In [6]:
the_prob_threshold = 0.03

# Computation

##### Create the scenario-based polyhedral uncertainty set object, which loads the data


In [7]:
# Create the scenario-based polyhedral uncertainty set object, which loads the data
the_scenarios_pus = spus.ScenariosPUS(the_input_file,a_n_values_per_attribute_per_data_point=the_n_values_per_attribute_per_data_point)

##### Step 3a → 3c: PCA reduction

In [8]:
the_attributes_to_merge_via_pca  = [the_attributes_to_merge_pca_level_1, the_attributes_to_merge_pca_level_2]
the_explained_variance_threshold = [{k:the_explained_variance_threshold_level_1 for k in the_attributes_to_merge_pca_level_1},{k:the_explained_variance_threshold_level_2 for k in the_attributes_to_merge_pca_level_2}] 
the_n_directions_threshold       = [{k:the_n_directions_threshold_level_1 for k in the_attributes_to_merge_pca_level_1},{k:the_n_directions_threshold_level_2 for k in the_attributes_to_merge_pca_level_2}] 

In [9]:
the_scenarios_pus.perform_size_reduction(the_attributes_to_merge_via_pca,the_explained_variance_threshold,the_n_directions_threshold)

##### Step 3d: clustering

In [10]:
the_scenarios_pus.perform_clustering(the_n_clusters,a_method=the_clustering_method)

/Users/nvdbconsulting/Documents/local_research/uncertainty_modelling_for_energy/myenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/nvdbconsulting/Documents/local_research/uncertainty_modelling_for_energy/myenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/nvdbconsulting/Documents/local_research/uncertainty_modelling_for_energy/myenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


##### Step 3e → 3f: PUS of each cluster of each new final attribute

In [11]:
the_scenarios_pus.compute_polyhedral_uncertainty_set_for_each_cluster_of_each_attribute(a_details_polyhedral_uncertainty_set_generation=the_details_polyhedral_uncertainty_set_generation)

##### Step 4: scenario definition

In [12]:
the_scenarios_pus.perform_scenario_definition(a_prob_threshold=the_prob_threshold)
print(f"The number of scenarios verifying the probability threshold {the_scenarios_pus.prob_threshold} is equal to {the_scenarios_pus.n_scenarios}.")

The number of scenarios verifying the probability threshold 0.03 is equal to 14.


# Querying the ouput results: example

##### Get the list of all scenarios generated

In [13]:
the_scenarios = the_scenarios_pus.get_scenarios()

##### Get the details on a specific scenario, here the scenario with index 'the_scenario_index'

In [14]:
the_scenario_index = 3

In [15]:
the_scenar = the_scenarios[the_scenario_index-1]

# Get the probability of the scenario
the_proba = the_scenar.get_probability()

# Get the detais on the linear constraints for generating uncertainty for the scenario
the_scenario_linear_formulation_details = the_scenar.get_linear_constraints_for_optimisation()

##### Print the details on the scenario with index 'the_scenario_index'

In [16]:
print(f"The probabilty of occurence of scenario {the_scenario_index} is {the_proba}.")

for attr in the_scenario_linear_formulation_details:

    the_scenario_linear_formulation_details_attr = the_scenario_linear_formulation_details[attr]
    
    print(f"\n\n--------------------------------------------------------")
    print(f" = Constraints for the final attribute '{attr}' =")
    print(f"--------------------------------------------------------")

    print("PUS linear constraints dimensions: A_w * w + A_z * z <= b")
    print(f"A_w: {the_scenario_linear_formulation_details_attr['A_w'].shape}")
    print(f"A_z: {the_scenario_linear_formulation_details_attr['A_z'].shape}")
    print(f"b: {the_scenario_linear_formulation_details_attr['b'].shape}")

    print("\nProjection from truncated basis to original basis: u = w' * A_to_original_basis + b_to_original_basis")
    print(f"A_to_original_basis: {the_scenario_linear_formulation_details_attr['A_to_original_basis'].shape}")
    print(f"b_to_original_basis: {the_scenario_linear_formulation_details_attr['b_to_original_basis'].shape}")

    print(f"\nDetails on how the original uncertainty vector is composed in the different attributes:\n{the_scenario_linear_formulation_details_attr['description_var_original_basis']}")

The probabilty of occurence of scenario 3 is 0.09.


--------------------------------------------------------
 = Constraints for the final attribute 'Seasonality' =
--------------------------------------------------------
PUS linear constraints dimensions: A_w * w + A_z * z <= b
A_w: (430, 48)
A_z: (430, 144)
b: (430,)

Projection from truncated basis to original basis: u = w' * A_to_original_basis + b_to_original_basis
A_to_original_basis: (48, 600)
b_to_original_basis: (600,)

Details on how the original uncertainty vector is composed in the different attributes:
[('Elec', 24), ('GasDem_EA', 24), ('GasDem_EM', 24), ('GasDem_NE', 24), ('GasDem_NO', 24), ('GasDem_NT', 24), ('GasDem_NW', 24), ('GasDem_SC', 24), ('GasDem_SE', 24), ('GasDem_SO', 24), ('GasDem_SW', 24), ('GasDem_WM', 24), ('Temp_EA', 24), ('Temp_EM', 24), ('Temp_NE', 24), ('Temp_NO', 24), ('Temp_NT', 24), ('Temp_NW', 24), ('Temp_SC', 24), ('Temp_SE', 24), ('Temp_SO', 24), ('Temp_SW', 24), ('Temp_WM', 24), ('Temp_WN', 24), 